In [1]:
import pandas as pd
from apyori import apriori
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [2]:
# Cargar el dataset
file_path = "movies.csv"
df = pd.read_csv(file_path, delimiter=';', encoding='ISO-8859-1')

# Mostrar las primeras filas para exploración
df.head()

,id,budget,genres,homePage,productionCompany,productionCompanyCountry,productionCountry,revenue,runtime,video,...,popularity,releaseDate,voteAvg,voteCount,genresAmount,productionCoAmount,productionCountriesAmount,actorsAmount,castWomenAmount,castMenAmount
0,5,4000000,Crime|Comedy,https://www.miramax.com/movie/four-rooms/,Miramax|A Band Apart,US|US,United States of America,4257354.0,98,False,...,20.880,9/12/1995,5.7,2077.0,2.0,2.0,1.0,25.0,15,9
1,6,21000000,Action|Thriller|Crime,NaN,Universal Pictures|Largo Entertainment|JVC,US|US|JP,Japan|United States of America,12136938.0,110,False,...,9.596,15/10/1993,6.5,223.0,3.0,3.0,2.0,15.0,3,9
2,11,11000000,Adventure|Action|Science Fiction,http://www.starwars.com/films/star-wars-episod...,Lucasfilm|20th Century Fox,US|US,United States of America,775398007.0,121,NaN,...,100.003,25/05/1977,8.2,16598.0,3.0,2.0,1.0,105.0,5,62
3,12,94000000,Animation|Family,http://movies.disney.com/finding-nemo,Pixar,US,United States of America,940335536.0,100,NaN,...,134.435,30/05/2003,7.8,15928.0,2.0,1.0,1.0,24.0,5,18
4,13,55000000,Comedy|Drama|Romance,NaN,Paramount|The Steve Tisch Company,US|,United States of America,677387716.0,142,False,...,58.751,6/07/1994,8.5,22045.0,3.0,2.0,1.0,76.0,18,48


In [10]:
# Crear rangos para cada variable numérica basada en los cuartiles
df['budget_range'] = pd.cut(df['budget'], bins=[-1, 0, 500000, 20000000, 380000000], 
                            labels=['Sin presupuesto', 'Presupuesto Bajo', 'Presupuesto Medio', 'Presupuesto Alto'])

df['revenue_range'] = pd.cut(df['revenue'], bins=[-1, 0, 160000, 40000000, 2800000000], 
                             labels=['Sin ingresos', 'ingresos Bajo', 'ingresos Medio', 'ingresos Alto'])

df['runtime_range'] = pd.cut(df['runtime'], bins=[-1, 90, 100, 110, 750], 
                             labels=['Duracion Corta', 'Duracion Media', 'Duracion Larga', 'Duracion Muy Larga'])

df['popularity_range'] = pd.cut(df['popularity'], bins=[-1, 14.00, 22.00, 40.00, 11000.00], 
                                labels=['Popularidad Baja', 'Popularidad Media', 'Popularidad Alta', 'Popularidad Muy Alta'])

df['voteAvg_range'] = pd.cut(df['voteAvg'], bins=[-1, 5.0, 6.5, 7.5, 9.5], 
                             labels=['Calificacion Baja', 'Calificacion Media', 'Calificacion Alta', 'Calificacion Muy Alta'])

df['actorsAmount_range'] = pd.cut(df['actorsAmount'], bins=[-1, 10, 25, 50, 100], 
                                  labels=['Pocos Actores', 'Normal Actores', 'Muchos Actores', 'Elenco Grande Actores'])

# Verificar que las nuevas columnas han sido creadas
display(df[['budget_range', 'revenue_range', 'runtime_range', 'popularity_range', 
            'voteAvg_range', 'actorsAmount_range']].head())


,budget_range,revenue_range,runtime_range,popularity_range,voteAvg_range,actorsAmount_range
0,Presupuesto Medio,ingresos Medio,Duracion Media,Popularidad Media,Calificacion Media,Normal Actores
1,Presupuesto Alto,ingresos Medio,Duracion Larga,Popularidad Baja,Calificacion Media,Normal Actores
2,Presupuesto Medio,ingresos Alto,Duracion Muy Larga,Popularidad Muy Alta,Calificacion Muy Alta,NaN
3,Presupuesto Alto,ingresos Alto,Duracion Media,Popularidad Muy Alta,Calificacion Muy Alta,Normal Actores
4,Presupuesto Alto,ingresos Alto,Duracion Muy Larga,Popularidad Muy Alta,Calificacion Muy Alta,Elenco Grande Actores


In [11]:
# Seleccionar las columnas categorizadas para Apriori
categorical_cols = ['budget_range', 'revenue_range', 'runtime_range', 
                    'popularity_range', 'voteAvg_range', 'actorsAmount_range']

# Convertir las categorías en strings (ya están en formato categoría) y reemplazar nulos con '0'
df[categorical_cols] = df[categorical_cols].astype(str).fillna('0')

# Convertir el DataFrame en una lista de listas (cada fila es una transacción)
records = df[categorical_cols].values.tolist()

# Mostrar el número de transacciones y algunos ejemplos
print(f"Total de transacciones procesadas: {len(records)}")
print("Ejemplo de transacciones:")

for i in range(5):  # Mostrar 5 ejemplos de transacciones
    print(records[i])


Total de transacciones procesadas: 10000
Ejemplo de transacciones:
['Presupuesto Medio', 'ingresos Medio', 'Duracion Media', 'Popularidad Media', 'Calificacion Media', 'Normal Actores']
['Presupuesto Alto', 'ingresos Medio', 'Duracion Larga', 'Popularidad Baja', 'Calificacion Media', 'Normal Actores']
['Presupuesto Medio', 'ingresos Alto', 'Duracion Muy Larga', 'Popularidad Muy Alta', 'Calificacion Muy Alta', 'nan']
['Presupuesto Alto', 'ingresos Alto', 'Duracion Media', 'Popularidad Muy Alta', 'Calificacion Muy Alta', 'Normal Actores']
['Presupuesto Alto', 'ingresos Alto', 'Duracion Muy Larga', 'Popularidad Muy Alta', 'Calificacion Muy Alta', 'Elenco Grande Actores']


**Primera generacion de reglas con 10% de soporte y 70% de confianza**

In [29]:
# Aplicar el algoritmo Apriori
reglas_asociacion = apriori(records, min_support=0.1, min_confidence=0.7)

# Convertir las reglas en una lista
reglas = list(reglas_asociacion)

# Mostrar el número de reglas encontradas
print(f"Total de reglas encontradas: {len(reglas)}")

print("\nReglas de asociación encontradas:\n")
for i, regla in enumerate(reglas):
    print(f"Regla {i+1}: {regla}\n")



Total de reglas encontradas: 13

Reglas de asociación encontradas:

Regla 1: RelationRecord(items=frozenset({'Duracion Corta', 'Sin ingresos'}), support=0.2086, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Duracion Corta'}), items_add=frozenset({'Sin ingresos'}), confidence=0.7321867321867322, lift=1.5327333728003605)])

Regla 2: RelationRecord(items=frozenset({'Sin presupuesto', 'Duracion Corta'}), support=0.1996, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Duracion Corta'}), items_add=frozenset({'Sin presupuesto'}), confidence=0.7005967005967007, lift=1.4562392446408243)])

Regla 3: RelationRecord(items=frozenset({'Pocos Actores', 'Sin ingresos'}), support=0.1634, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Pocos Actores'}), items_add=frozenset({'Sin ingresos'}), confidence=0.8659247482776894, lift=1.8126957259319434)])

Regla 4: RelationRecord(items=frozenset({'Pocos Actores', 'Sin presupuesto'}), support=0.16, ordered_statistics=[Order

**Segunda generacion de reglas con 20% de soporte y 70% de confianza**

In [30]:
# Aplicar el algoritmo Apriori
reglas_asociacion = apriori(records, min_support=0.2, min_confidence=0.7)

# Convertir las reglas en una lista
reglas = list(reglas_asociacion)

# Mostrar el número de reglas encontradas
print(f"Total de reglas encontradas: {len(reglas)}")

print("\nReglas de asociación encontradas:\n")
for i, regla in enumerate(reglas):
    print(f"Regla {i+1}: {regla}\n")


Total de reglas encontradas: 2

Reglas de asociación encontradas:

Regla 1: RelationRecord(items=frozenset({'Duracion Corta', 'Sin ingresos'}), support=0.2086, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Duracion Corta'}), items_add=frozenset({'Sin ingresos'}), confidence=0.7321867321867322, lift=1.5327333728003605)])

Regla 2: RelationRecord(items=frozenset({'Sin presupuesto', 'Sin ingresos'}), support=0.395, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Sin ingresos'}), items_add=frozenset({'Sin presupuesto'}), confidence=0.8268787942223152, lift=1.7187254089010917), OrderedStatistic(items_base=frozenset({'Sin presupuesto'}), items_add=frozenset({'Sin ingresos'}), confidence=0.8210351278320516, lift=1.7187254089010917)])



**Tercera generacion de reglas con 15% de soporte y 80% de confianza**

In [33]:
# Aplicar el algoritmo Apriori
reglas_asociacion = apriori(records, min_support=0.15, min_confidence=0.8)

# Convertir las reglas en una lista
reglas = list(reglas_asociacion)

# Mostrar el número de reglas encontradas
print(f"Total de reglas encontradas: {len(reglas)}")

print("\nReglas de asociación encontradas:\n")
for i, regla in enumerate(reglas):
    print(f"Regla {i+1}: {regla}\n")


Total de reglas encontradas: 6

Reglas de asociación encontradas:

Regla 1: RelationRecord(items=frozenset({'Pocos Actores', 'Sin ingresos'}), support=0.1634, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Pocos Actores'}), items_add=frozenset({'Sin ingresos'}), confidence=0.8659247482776894, lift=1.8126957259319434)])

Regla 2: RelationRecord(items=frozenset({'Pocos Actores', 'Sin presupuesto'}), support=0.16, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Pocos Actores'}), items_add=frozenset({'Sin presupuesto'}), confidence=0.8479067302596714, lift=1.7624334447301422)])

Regla 3: RelationRecord(items=frozenset({'Sin presupuesto', 'Sin ingresos'}), support=0.395, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Sin ingresos'}), items_add=frozenset({'Sin presupuesto'}), confidence=0.8268787942223152, lift=1.7187254089010917), OrderedStatistic(items_base=frozenset({'Sin presupuesto'}), items_add=frozenset({'Sin ingresos'}), confidence=0.821035127832

# Hallazgos y conclusiones


1. Distribución de Presupuestos y Ingresos:
La mayoría de las películas en el dataset tienen un presupuesto bajo o medio, lo que sugiere que las producciones de alto presupuesto son menos comunes pero son las que generan ingresos altos.
Los ingresos también muestran una tendencia similar, con muchas películas generando ingresos bajos. Esto puede indicar que las películas de bajo presupuesto a menudo no logran un gran éxito en taquilla.

2. Rango de Duración:
La duración de las películas se distribuye principalmente entre las categorías de duración corta y media. Esto puede reflejar una preferencia del público por películas que no son excesivamente largas.

3. Popularidad:
La mayoría de las películas caen en la categoría de popularidad baja a media. Esto sugiere que hay un número significativo de películas que no logran captar la atención del público.

4. Calificaciones:
Las calificaciones promedio de las películas tienden a estar en el rango medio, lo que puede indicar que, aunque hay películas bien recibidas, también hay muchas que no alcanzan una buena crítica.

5. Cantidad de Actores:
La mayoría de las películas tienen un número moderado de actores, lo que puede ser indicativo de producciones que no dependen de grandes elencos para atraer a la audiencia.

6. Reglas de Asociación:
Las reglas de asociación generadas a partir del algoritmo Apriori revelan patrones interesantes entre las diferentes categorías. Por ejemplo, puede haber asociaciones entre películas de ciertos géneros y rangos de presupuesto, lo que podría ayudar a los productores a identificar qué tipos de películas tienen más probabilidades de ser exitosas en función de su presupuesto y género.

7. Recomendaciones para Productores:
Basado en los hallazgos, se recomienda a los productores que consideren invertir en películas con presupuestos moderados y que se enfoquen en géneros que han demostrado ser populares en el pasado.

8. Oportunidades de Mejora:
Se sugiere realizar un análisis más profundo sobre las características de las películas que han tenido éxito en taquilla, así como investigar las razones detrás de la baja popularidad de muchas producciones.

Conclusión:
El análisis de los datos de películas proporciona una visión valiosa sobre las tendencias actuales en la industria cinematográfica. Las reglas de asociación y los patrones observados pueden servir como herramientas útiles para la toma de decisiones estratégicas en la producción y marketing de películas viendo las distintas categorías.